In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import os, time, pickle
from scripts.tools.issuetracker import IssuetrackerAPI
from scripts.tools.issuetracker.issues import Issues
from jpnotebooks.Software.frs_tools import extract, matchers
from jpnotebooks.Software.frs_tools.extract import *
from officelib import xllib
from officelib.const import xlconst as xlc
from officelib.wordlib import Word
from officelib.wordlib import c as wdc
from officelib import wordlib
from pythoncom import com_error
from officelib import const as wdc

In [12]:
def expired(loaded, expires):
    now = time.time()
    age = (now-loaded)/3600
    return age > expires

class IssueCache():
    def __init__(self, issues=None, dltime=None, expires=12, api=None):
        self.issues = None
        self._parse_issues(issues, api)
        self.loaded = dltime or time.time()
        self.expires = expires
        
    def _parse_issues(self, issues, api):
        if isinstance(issues, dict):
            self.issues = Issues(api, list(issues.values()))
        elif issues:
            self.issues = Issues(api, list(issues))
            
    def age(self):
        return self._age(self.loaded)
    
    def get(self):
        return self.issues.copy()
    
    @classmethod
    def from_dl(cls, api, *a, expires=12, **k):
        s = cls(expires=expires)
        s.reload(api, *a, **k)
        return s
    
    def reload(self, api, *a, **k):
        api.maybe_login()
        iss = api.download_issues(*a, **k)
        self._parse_issues(iss, api)
        self.loaded = time.time()
    
    def _age(self, t):
        now = time.time()
        hrs = (now - t)/3600
        return hrs        
    
    def expired(self):
        return self.age() > self.expires
    
    def save(self, fn):
        with open(fn, 'wb') as f:
            pickle.dump((self.loaded, self.issues), f)
            
    @classmethod
    def open(cls, fn, api, *a, expires=12, **k):
        try:
            s = cls.load(fn, expires)
        except (FileNotFoundError, EOFError, pickle.PickleError):
            s = cls(None, -1)  # trigger expired
            
        if s.expired():
            s.reload(api, *a, **k)
            s.save(fn)
        return s
        
    @classmethod
    def load(cls, fn, expires=12):
        with open(fn, 'rb') as f:
            ob = pickle.load(f)
        loaded, issues = ob
        return cls(issues, loaded, expires)     

In [37]:
import sys
class FormattingError(Exception):
    pass

class ReleaseNotesFormatter():
    def __init__(self, issues, header=False):
        self.issues = issues
        self.release_notes = []
        self.sections = []
        self.include_header = header
        self.parse()
        
    def check(self, **kw):
        if not self.sections:
            self._extract()
        
        def match(i):
            for k,v in kw.items():
                if isinstance(v, tuple):
                    if getattr(i, k) not in v:
                        return False
                else:
                    if getattr(i, k) != v:
                        return False
            return True
        
        iset = {i.id for i in self.issues if match(i)}
        iset2 = {t[0].id for t in self.sections}
        diff = iset - iset2
        if diff:
            print("Issues without Release Notes:")
            ldiff = sorted(diff)
            for i in ldiff:
                print(" ", i)
            return ldiff
        return []
        
    def _extract(self):
        extractor = extract.IssuesExtractor(self.issues)
        self.sections = extractor.extract("Release Notes")
        
    def parse(self):
        self._extract()
        self.release_notes = rnl = []
        for issue, notes in self.sections:
            if notes.lower() == "* n/a":
                continue
            if self.include_header:
                rns = "%s:\n%s"%(_header(issue), notes)
            else:
                rns = notes
            rnl.append(rns)
            
    def write(self, b):
        b.write("\n".join(self.release_notes))
        
    def write2(self, fn):
        with open(fn, 'w') as f:
            self.write(f)
        
    def print(self):
        self.write(sys.stdout)
        
    def parse2(self):
        self.parse()
        self.print()
        
    def categorize(self, *cat):
        # attempt to categorize issues by subject based on provided categories
        self.release_notes = rnl = []
        cat = [c.lower() for c in cat]
        cats = {c: [] for c in cat}
        cats["other"] = []
        cats["Bug Fixes"] = []
        for iss, notes in self.sections:
            key = None
            if iss.tracker == "Bug":
                key = "Bug Fixes"
            else:
                sub = iss.subject.lower()
                for c in cat:
                    if c in sub:
                        key = c
                        break
                else:
                    key = "other"
            if notes.lower() == "* n/a":
                continue
            lines = notes.splitlines()
            lines2 = []
            for l in lines:
                stars, note = l.split(" ", 1)
                note = "    "*(len(stars)) + "* " + note
                lines2.append(note)
            cats[key].append(lines2)
        
        for category, notes in cats.items():
            rnl.append("\n%s:"%category.title())
            for inotes in notes:
                for line in inotes:
                    rnl.append(line)
                    
    def categorize2(self, *cat):
        # attempt to categorize issues by subject based on provided categories
        self.release_notes = rnl = []
        for iss, notes in self.sections:
            key = None
            if notes.lower() == "* n/a":
                continue
            if iss.tracker == "Bug":
                key = "bug fixes"
            else:
                sub = iss.subject.lower()
                for c in cat:
                    c = c.lower()
                    if c in sub:
                        key = c
                        break
                else:
                    key = "other"
            lines = notes.splitlines()
            for l in lines:
                stars, note = l.split(" ", 1)
                note = "|".join((key, str(iss.id), stars, note))
                rnl.append(note)
                
    def xlify(self):
        self.release_notes = rnl = []
        for iss, notes in self.sections:
            if notes.lower() == "* n/a": 
                continue
            if iss.tracker == "Bug":
                cat1 = "bug"
            else:
                cat1 = "other"
            lines = notes.splitlines()
            for l in lines:
                stars, note = l.split(" ", 1)
                if any(s != "*" for s in stars):
                    raise FormattingError("Note without indent for issue #%d: '%s'"%(iss.id, l))
                note = cat1, "", str(iss.id), stars, note
                rnl.append(note)
                    
def _header(issue):
    return "Issue #%d - %s - %s"%(issue.id, issue.tracker, issue.subject)

In [106]:
CACHE_FILE = ".\\.issuetracker\issue_cache.pkl"

def invalidate_cache():
    if os.path.exists(CACHE_FILE):
        os.remove(CACHE_FILE)

def _fixup(c1):
    """
    Fix up indents and italics.
    """
    while c1.Value:
        ind = len(c1.Value)-1
        c2 = c1.Offset(1,2)
        c2.IndentLevel = ind*2
        v = c2.Value
        if "_" in v:
            c2.Value = v.replace("_","")
        c1 = c1.Offset(2,1)

def rn2xl(formatter, sm):
    xl, wb, ws, cells = xllib.xlObjs()
    with xllib.screen_lock(xl):
        cr = cells.Range
        c1 = cr("A1")
        data = formatter.release_notes
        c2 = c1.Offset(len(data), len(data[0]))
        print("Pasting data...")
        cr(c1,c2).Value = data
        print("Performing cleanup")
        _fixup(cr("A1").Offset(1, len(data[0])-1))
        ws.Name = "RN_RAW"
        if isinstance(sm, tuple):
            sm = " ".join(sm)
        saveas(wb, "%s Release Notes.xlsx"%sm)
        return xl,wb,ws

def create_rn_spreadsheet(sprint_milestone):
    """
    sprint_milestone: string of sprint or tuple of sprints to choose from
    """
    print("Connecting to API instance")
    api = IssuetrackerAPI("issue.pbsbiotech.com", "nstarkweather@pbsbiotech.com", "kookychemist", login=False)
    cache = IssueCache.open(CACHE_FILE, api, "pbssoftware", expires=1)
    print("Collecting issues from the following milestone(s): %s"%str(sprint_milestone))
    issues = Issues(api, cache.get())
    ilist = issues.find2(sprint_milestone=sprint_milestone)
    formatter = ReleaseNotesFormatter(ilist, False)
    print("Formatting issues for Excel import")
    formatter.xlify()
    return rn2xl(formatter, sprint_milestone)

def collect_data(ws):
    cr = ws.Cells.Range
    c1 = cr("A1")
    # cat2 might be blank, so start toright at 3rd column
    c2 = c1.Offset(1,3).End(xlc.xlToRight).End(xlc.xlDown)
    return cr(c1, c2).Value
    
from collections import OrderedDict

def _lookup(map, key):
    return map.get(key, (key or "").title())

def _innerorder(rows, map, out, order=None):
    if not rows:
        return
    
    cat = _lookup(map, rows[0][0])
    o2s = set(); o2s.add("")
    o2l = order if order else [""]
    if o2l[0] != "":
        o2l.insert(0, "")
    for r in rows:
        c1 = _lookup(map, r[0])
        c2 = _lookup(map, r[1])
        if cat != c1:
            raise FormattingError("Bad row sorting: %s %s"%(cat, c1))
        if c2 not in o2s:
            o2s.add(c2)
            o2l.append(c2)
    
    # second pass - write rows
    i = 0  # for sanity check 
    for o in o2l:
        for r in rows:
            if _lookup(map, r[1]) == o:
                out.append(r)
                i += 1
                
    if i != len(rows):  
        raise FormattingError("Mismatching inner sort row length: %s != %s"%(i, len(rows)))
        
def order_notes(data, order=(), map=None, suborder=None):
    out = []
    map = map or {}
    suborder = suborder or {}
    
    # the category mapping is more useful than spreadsheet names
    # for organizing. this is only necessary because two spreadsheet
    # names can map to the same category, e.g. bugfix -> Bug and bug -> Bug
    # this wouldn't be necessary if i was more strict about the names
    # but whatever. Order and Suborder are converted to their category
    # names here to be more useful. 
    
    # the reason to split naming into category vs spreadsheet names is 
    # just because the category names might be kind of obnoxious to write
    # out correctly in the spreadsheet, since the string lookup requires
    # an exact match. E.g. easier to write "gmp" than "GMP and CFR 21p11 compliance"
    # and make sure all punctuation is correct, etc.....
    
    # note that order is compressed into the unique category names, but
    # suborder has to merge any lists together.
    
    print("Fixing up order and suborder mappings...")
    suborder2 = {}
    for o, v in suborder.items():
        cat = _lookup(map, o)
        if cat not in suborder2:
            suborder2[cat] = [""]
        if "" in v:
            v.remove("")
        suborder2[cat].extend(v)
        
    order2 = [_lookup(map, o) for o in order]
    
    print("Performing sort")
    for o in order2:
        rows = []
        for row in data:
            if o == _lookup(map, row[0]):
                rows.append(row)
        _innerorder(rows, map, out, suborder2.get(o, None))
    
    # extra sanity checks
    print("Sanity checking the resulting list...")
    if len(data) != len(out):
        raise FormattingError("oops")
    if set(data) != set(out):
        raise FormattingError("oops2")
    return out

def reinsert_data(wb, data, ws_name):
    print("Inserting sorted data into worksheet")
    with xllib.screen_lock(wb.Application):  # hide alert for deleting worksheet
        for ws2 in wb.Worksheets:
            if ws2.Name == ws_name:
                ws2.Delete()
                break
        ws2 = wb.Worksheets.Add()
        ws2.Name = ws_name
        cr2 = ws2.Cells.Range
        c1 = cr2("A1")
        c2 = c1.Offset(len(data), len(data[0]))
        cr2(c1, c2).Value = data
        _fixup(cr2("A1").Offset(1, len(data[0])-1))

In [107]:
def wordify(data, map):
    print("Preparing data for word...")
    dat = OrderedDict()
    for cat1, cat2, i, stars, note in data:
        c1 = _lookup(map, cat1)
        c2 = _lookup(map, cat2)
        if c1 not in dat:
            dat[c1] = OrderedDict()
        if c2 not in dat[c1]:
            dat[c1][c2] = []
        dat[c1][c2].append((len(stars), note))
    return dat

In [113]:
def make_style(doc, name="PY_RN_TITLE", font_size=24, font="Calibri", 
                     alignment=wdc.wdAlignParagraphLeft, indent=None, tabstops=None, space_after=None):
    
    if alignment is None:
        alignment = wordlib.c.wdAlignParagraphCenter
    
    style = doc.Styles.Add(name)
    style.BaseStyle = doc.Styles("Normal")
    style.NoSpaceBetweenParagraphsOfSameStyle = True
    
    style.Font.Size = font_size
    style.Font.Name = font
    
    style.ParagraphFormat.Alignment = alignment
    style.ParagraphFormat.SpaceBeforeAuto = False
    style.ParagraphFormat.SpaceAfterAuto = False
    style.ParagraphFormat.LineSpacingRule = wordlib.c.wdLineSpaceSingle
    
    if indent is not None:
        style.ParagraphFormat.LeftIndent = wordlib.inches_to_points(indent)
    
    if tabstops is not None:
        style.ParagraphFormat.TabStops.Add(wordlib.inches_to_points(tabstops), 
                                           wordlib.c.wdAlignTabLeft, 
                                           wordlib.c.wdTabLeaderDots)
    
    if space_after is not None:
        style.ParagraphFormat.SpaceAfter = space_after
    
    return style

_styles = {}
def get_style(d, indent, cat):
    n = len(indent)
    i = n
    if cat is not None:
        i += 1
    name = "PY_RN_BULLET%d_%d"%(i, n)
    style = _styles.get(name)
    if style is None:
        style = bulletstyle(d, name, n, 9)
        _styles[name] = style
    return style

def move(r,n):
    r.MoveStart(wdc.wdCharacter, n)
    
def pwrite(r, txt, style):
    if isinstance(style, str):
        style = _styles[style]
    r.Text = txt
    r.Style = style
    # check for "note", "dev note"
    l = txt.split(":",1)[0].lstrip("* ").lower()
    if l == "note" or l == "developer note":
        r.Italic=True
        r2 = r.Document.Range(Start=r.Start, End=r.Start+txt.find(":")+1)
        r2.Bold=True
    r.InsertAfter("\r")
    move(r, len(txt)+1)
    
def _unpack_styles(d):
    for style in d.Styles:
        _styles[style.NameLocal] = style
            
def _normal_style(d):
    style = make_style(d, "PY_RN_NORMAL", 9, 0)
    _styles[style.NameLocal] = style
    
def clear(d):
    r = d.Paragraphs(1).Range
    r.MoveEnd(wdc.wdStory, 1)
    r.Text = ""
    r.Style = "Normal"
    
def word_out(w, d, data):
    clear(d)
    _unpack_styles(d)
    pgs = d.Paragraphs
    r = pgs(1).Range
    
    # summary first
    l2 = data.get("Summary", {None: [(0, "See below for list of changes.")]})
    pwrite(r, "%s:"%"Summary", "PY_RN_CAT1")
    for _, lines in l2.items():
        for _, line in lines:
            pwrite(r, line, "PY_RN_BULLET_1_0")
    
    # all items except 'Summary' and 'Exclude'
    for cat1, l2 in data.items():
        if cat1 == "Summary" or cat1 == "Exclude":
            continue
        pwrite(r, "%s:"%cat1, 'PY_RN_CAT1')
        for cat2, l2 in l2.items():
            i = int(bool(cat2))
            if cat2 != "" and cat2 is not None: 
                pwrite(r, "%s:"%cat2, 'PY_RN_CAT2')
            for n, note in l2:
                pwrite(r,note, 'PY_RN_BULLET_%d_%d'%(n,i))
    r.Delete()
    
def create_word_doc(data, template):
    w = Word()
    print("Exporting files to word...")
    with wordlib.screen_lock(w):
        d = w.Documents.Open(template, ReadOnly=True)
        word_out(w, d, data)
    return w,d
        
def saveas(d, name):
    path = os.path.abspath(name)
    print("Saving as '%s'"%path)
    d.SaveAs(path)

In [114]:
def download_new(sprint):
    return create_rn_spreadsheet(sprint)

def open_existing(file):
    xl, wb = xllib.xlBook2(file)
    ws = wb.Worksheets("RN_RAW")
    return xl, wb, ws

In [115]:
mksprint("3.0.1")

Connecting to API instance
Formatting issues for Excel import
Pasting data...
Performing cleanup
Saving as 'C:\Users\Nathan\Documents\Personal\Test\3.0.1 Release Notes.xlsx'
Fixing up order and suborder mappings...
Performing sort
Sanity checking the resulting list...
Inserting sorted data into worksheet
Preparing data for word...
Exporting files to word...
Saving as 'C:\Users\Nathan\Documents\Personal\Test\3.0.1 Release Notes.docx'


In [116]:
mksprint("3.0.2")

Connecting to API instance
Formatting issues for Excel import
Pasting data...
Performing cleanup
Saving as 'C:\Users\Nathan\Documents\Personal\Test\3.0.2 Release Notes.xlsx'
Fixing up order and suborder mappings...
Performing sort
Sanity checking the resulting list...
Inserting sorted data into worksheet
Preparing data for word...
Exporting files to word...
Saving as 'C:\Users\Nathan\Documents\Personal\Test\3.0.2 Release Notes.docx'


In [117]:
mksprint("3.0.3")

Connecting to API instance
Formatting issues for Excel import
Pasting data...
Performing cleanup
Saving as 'C:\Users\Nathan\Documents\Personal\Test\3.0.3 Release Notes.xlsx'
Fixing up order and suborder mappings...
Performing sort
Sanity checking the resulting list...
Inserting sorted data into worksheet
Preparing data for word...
Exporting files to word...
Saving as 'C:\Users\Nathan\Documents\Personal\Test\3.0.3 Release Notes.docx'


In [29]:
# -------------------------------------------
# Step 1 (option 1) - Generate the spreadsheet
# -------------------------------------------

xl, wb, ws = download_new("3.0")

# -------------------------------------------
# Step 1 (option 2)- Open existing spreadsheet
# -------------------------------------------

#xlfile = os.path.abspath("release notes test 3.0 categorize.xlsx")
#xl, wb, ws = open_existing(xlfile)

In [ ]:
# Now manually adjust categories and text as as desired in CSV format !!!
_=input("Adjust spreadsheet categories manually !!!")

In [32]:
# -------------------------------------------
# Step 1.5 - Collect data from spreadsheet
# -------------------------------------------

data = collect_data(ws)

# -------------------------------------------
# Step 2 (option 1)- order all of the categories correctly and create a new worksheet with the ordered data
# -------------------------------------------

# order of categories (spreadsheet names)
order = "summary shell webui gmp controls alarms harvest misc bugfix exclude".split()

# name mapping spreadsheet name -> category name
# applies to both cat1 and cat2!
cat_map = {
    "webui": "Hello UI",
    "bugfix": "Bug Fixes",
    "misc": "Miscellaneous",
    "other": "Miscellaneous",
    "gmp": "GMP and CFR 21 part 11 Compliance",
    "shell": "Desktop UI",
    "harvest": "Harvest Mode (15L Mag and larger only)",
    "controls": "Controls",
    "permissions": "Permissions",
    "data_and_reports": "Data and Reports",
    "security": "Security",
    "alarms": "Alarms",
    'improvements': "Improvements",
    "bug": "Bug Fixes",
    "exclude": "Exclude",
    "summary": "Summary"
}

# mapping order for subcategories
suborder = {"bug": ("webui", "shell")}

# perform ordering
ordered_data = order_notes(data, order, cat_map, suborder)
reinsert_data(wb, ordered_data, "RN_SORTED")


# -------------------------------------------
# Step 2 (option 2)- collect data from existing spreadsheet
# -------------------------------------------

# ordered_data = collect_data(wb.Worksheets("RN_SORTED"))


# -------------------------------------------
# Step 4 - prepare the list for wordification
#          the easiest format to use is a nested OrderedDict
#          cat1 -> cat2 -> list of notes
#          uncomment the collect_data call if using an
#          existing worksheet.
# -------------------------------------------

word_data = wordify(ordered_data, cat_map)

# -------------------------------------------
# Step 5 - insert data into word and save. 
#          note that the calling function ABSOLUTELY REQUIRES
#          the use of a template document with prepared, properly
#          formatted styles, due to my (current) inability to 
#          dynamically generate arbitrarily nested list styles properly
#          through VBA. 
#          be sure to update the file path to the template accordingly!
# -------------------------------------------


w,d = create_word_doc(word_data, os.path.abspath("Release Notes Template5.docx"))
saveas(d, "Hello v3.0 Release Notes Draft 2 180226.docx")

Fixing up order and suborder mappings...
Performing sort
Sanity checking the resulting list...
Inserting sorted data into worksheet
Preparing data for word...
Exporting files to word...
Saving as 'C:\Users\Nathan\Documents\Personal\Test\Hello v3.0 Release Notes Draft 2 180226.docx'


### The following three cells show the above code without the comments. For brevity, the order and mapping containers are left empty. These cells are mostly for illustration.

In [ ]:
xl, wb, ws = open_existing("path\\to\\file\\file.xlsx")

In [ ]:
# edit spreadsheet here

In [ ]:
data = collect_data(ws)
order    = []
cat_map  = {}
suborder = {}
ordered_data = order_notes(data, order, cat_map, suborder)
reinsert_data(wb, ordered_data, "RN_SORTED")
word_data = wordify(ordered_data, cat_map)
w,d = create_word_doc(word_data, os.path.abspath("Release Notes Template5.docx"))
saveas(d, "My Release Notes.docx")

In [27]:
order = "summary shell webui gmp controls alarms harvest misc bugfix exclude".split()

cat_map = {
    "webui": "Hello UI",
    "bugfix": "Bug Fixes",
    "misc": "Miscellaneous",
    "other": "Miscellaneous",
    "gmp": "GMP and CFR 21 part 11 Compliance",
    "shell": "Desktop UI",
    "harvest": "Harvest Mode (15L Mag and larger only)",
    "controls": "Controls",
    "permissions": "Permissions",
    "data_and_reports": "Data and Reports",
    "security": "Security",
    "alarms": "Alarms",
    'improvements': "Improvements",
    "bug": "Bug Fixes",
    "exclude": "Exclude",
    "summary": "Summary"
}

# mapping order for subcategories
suborder = {"bug": ("webui", "shell")}

In [31]:
for sprint in ("3.0.1", "3.0.2", "3.0.3"):
    xl, wb, ws = download_new(sprint)
    data = collect_data(ws)
    # order    = []
    # cat_map  = {}
    # suborder = {}
    ordered_data = order_notes(data, order, cat_map, suborder)
    reinsert_data(wb, ordered_data, "RN_SORTED")
    word_data = wordify(ordered_data, cat_map)
    w,d = create_word_doc(word_data, os.path.abspath("Release Notes Template5.docx"))
    saveas(d, "%s Release Notes.docx"%sprint)

Connecting to API instance
Formatting issues for Excel import
Pasting data...
Performing cleanup
Fixing up order and suborder mappings...
Performing sort
Sanity checking the resulting list...
Inserting sorted data into worksheet
Preparing data for word...
Exporting files to word...
Saving as 'C:\Users\Nathan\Documents\Personal\Test\3.0.1 Release Notes.docx'
Connecting to API instance
Formatting issues for Excel import


FormattingError: Note without indent: '#### A constant, where it should technically be calculated before each time it runs, since changes in processor load can contribute to slight changes in the amount of time between running the control loop'

In [40]:
def mksprint(sprint):
    xl, wb, ws = download_new(sprint)
    data = collect_data(ws)
    # order    = []
    # cat_map  = {}
    # suborder = {}
    ordered_data = order_notes(data, order, cat_map, suborder)
    reinsert_data(wb, ordered_data, "RN_SORTED")
    word_data = wordify(ordered_data, cat_map)
    w,d = create_word_doc(word_data, os.path.abspath("Release Notes Template5.docx"))
    saveas(d, "%s Release Notes.docx"%sprint)
    wb.Save()

In [94]:
r=w.Selection.Range

In [63]:
r.Bold=True

In [95]:
r.Italic

0